In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

len(np.unique(housing['target']))

3842

In [2]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],
    "model__n_neurons": np.arange(1, 100),
    "model__learning_rate":  reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
    "model__optimizer": ['adam','sgd', 'nesterov'] 
}

In [3]:
def build_model(n_hidden, n_neurons, optimizer, learning_rate):
    if optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    if optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    model = tf.keras.models.Sequential()
    for i in range(n_hidden):
        if i == 0:
            model.add(tf.keras.layers.Dense(n_neurons, activation="relu", input_shape=X_train.shape[1:]))
        else:
            model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
    return model

In [4]:
import scikeras
from scikeras.wrappers import KerasRegressor
import tensorflow as tf

es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [5]:
from sklearn.model_selection import RandomizedSearchCV

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), verbose=0)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 15: early stopping
121/121 [==============================] - 1s 3ms/step
[CV] END model__learning_rate=0.009391503935379304, model__n_hidden=2, model__n_neurons=43, model__optimizer=sgd; total time=  21.0s
Epoch 11: early stopping
121/121 [==============================] - 1s 3ms/step
[CV] END model__learning_rate=0.009391503935379304, model__n_hidden=2, model__n_neurons=43, model__optimizer=sgd; total time=  14.8s
Epoch 15: early stopping
121/121 [==============================] - 1s 3ms/step
[CV] END model__learning_rate=0.009391503935379304, model__n_hidden=2, model__n_neurons=43, model__optimizer=sgd; total time=  21.1s
[CV] END model__learning_rate=0.005633126147948463, model__n_hidden=0, model__n_neurons=29, model__optimizer=nesterov; total time=   0.0s
[CV] END model__learning_rate=0.005633126147948463, model__n_hidden=0, model__n_neurons=29, model__optimizer=nesterov; total time=   0.0s
[CV] END model__learning

C:\Users\Dawid\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
9 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Dawid\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Dawid\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py", line 762, in fit
    self._fit(
  File "C:\Users\Dawid\AppData\Roaming\Python\Python39\site-packages\scikeras\wrappers.py", line 929, in _fit
    self._check_model_compatibility(y)
  F

Epoch 11: early stopping


RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x0000020A40D45250>], model=<function build_model at 0x0000020A3629DF70>),
                   param_distributions={'model__learning_rate': [0.0010360806612050577,
                                                                 0.007491073247979924,
                                                                 0.0014350969302655335,
                                                                 0.003287871559576513,
                                                                 0.0004056476384122555,
                                                                 0.0021272421770163774,
                                                                 0.00101...
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'model__optimizer': ['adam', 'sgd',
                                                             'nesterov']},
                   verbose=2)

In [6]:
import pickle

best_params = rnd_search_cv.best_params_

with open('rnd_search_params.pkl', 'wb') as f:
    pickle.dump(best_params, f)

with open('rnd_search_scikeras.pkl', 'wb') as f:
    pickle.dump(rnd_search_cv, f)

INFO:tensorflow:Assets written to: C:\Users\Dawid\AppData\Local\Temp\tmpn1_m98r9\assets


INFO:tensorflow:Assets written to: C:\Users\Dawid\AppData\Local\Temp\tmpn1_m98r9\assets


In [7]:
import keras_tuner as kt

def build_model_kt(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=3, default=2)
    n_neurons = hp.Int("n_neurons", min_value=1, max_value=100)
    learning_rate = hp.Float("learning_rate", min_value=3e-4, max_value=3e-2, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam", "nesterov"])
    
    if optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    if optimizer == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
    
    model = tf.keras.models.Sequential()
    for i in range(n_hidden):
        if i == 0:
            model.add(tf.keras.layers.Dense(n_neurons, activation="relu", input_shape=X_train.shape[1:]))
        else:
            model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
    return model

In [8]:
import os

In [10]:
random_search_tuner = kt.RandomSearch(build_model_kt, objective="val_mse", max_trials=10, overwrite=True,
                                      directory="my_california_housing", project_name="my_rnd_search", seed=42, max_consecutive_failed_trials=1000)
random_search_tuner.search(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[es])

root_logdir = os.path.join(random_search_tuner.project_dir, 'tensorboard')
tb = tf.keras.callbacks.TensorBoard(root_logdir)

best_params = {
    'n_hidden': random_search_tuner.get_best_hyperparameters(1)[0].get('n_hidden'),
    'n_neurons': random_search_tuner.get_best_hyperparameters(1)[0].get('n_neurons'),
    'learning_rate': random_search_tuner.get_best_hyperparameters(1)[0].get('learning_rate'),
    'optimizer': random_search_tuner.get_best_hyperparameters(1)[0].get('optimizer'),
}

with open('kt_search_params.pkl', 'wb') as f:
    pickle.dump(best_params, f)

best_model = random_search_tuner.get_best_models(1)[0]
best_model.save('kt_best_model.h5')

Trial 10 Complete [00h 00m 34s]

Best val_mse So Far: None
Total elapsed time: 00h 05m 55s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


In [11]:
best_params

{'n_hidden': 2,
 'n_neurons': 4,
 'learning_rate': 0.001968761070571233,
 'optimizer': 'sgd'}